In [1]:
#r "nuget:Microsoft.ML,1.5.1"
#r "nuget:Microsoft.ML.Recommender"
    
using System;
using System.Collections.Generic;
using System.Drawing;
using System.IO;
using System.Text;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.1

Installed package Microsoft.ML.Recommender version 0.17.1

In [2]:
public class MagenicSkill
{
    [LoadColumn(0)]
    public String PersonnelNumber;
    [LoadColumn(2)]
    public float SkillId;
    [LoadColumn(3)]
    public float SkillLevel;
}

public class SkillPrediction
{
    public String PersonnelNumber;
    public float Score;
}

In [3]:
MLContext mlContext = new MLContext();

var dataPath = Path.Combine(Environment.CurrentDirectory, "Data", "Magenic_Min_Skills.csv");

IDataView dataView = mlContext.Data.LoadFromTextFile<MagenicSkill>(dataPath, hasHeader: true, separatorChar: ',');

var requestedColumns = new List<DataViewSchema.Column>();

var cursor = dataView.GetRowCursorSet(requestedColumns, 0);

Console.WriteLine(cursor);

var split = mlContext.Data
    .TrainTestSplit(dataView, testFraction: 0.2,
    samplingKeyColumnName: "PersonnelNumber");

var trainSet = mlContext.Data
    .CreateEnumerable<MagenicSkill>(split.TrainSet, reuseRowObject: false);

var testSet = mlContext.Data
    .CreateEnumerable<MagenicSkill>(split.TestSet,reuseRowObject: false);
    
var trainLoader = mlContext.Data.LoadFromEnumerable<MagenicSkill>(trainSet);

var testLoader = mlContext.Data.LoadFromEnumerable<MagenicSkill>(testSet);

Microsoft.ML.DataViewRowCursor[]


In [4]:
public static void ConsoleWriteHeader(params string[] lines)
{
    var defaultColor = Console.ForegroundColor;
    Console.ForegroundColor = ConsoleColor.Yellow;
    Console.WriteLine(" ");
    foreach (var line in lines)
    {
        Console.WriteLine(line);
    }
    var maxLength = lines.Select(x => x.Length).Max();
    Console.WriteLine(new string('#', maxLength));
    Console.ForegroundColor = defaultColor;
}

public static void PeekDataViewInConsole(MLContext mlContext, IDataView dataView, IEstimator<ITransformer> pipeline, int numberOfRows = 4)
{
    string msg = string.Format("Peek data in DataView: Showing {0} rows with the columns", numberOfRows.ToString());
    ConsoleWriteHeader(msg);

    //https://github.com/dotnet/machinelearning/blob/master/docs/code/MlNetCookBook.md#how-do-i-look-at-the-intermediate-data
    var transformer = pipeline.Fit(dataView);
    var preparedData = transformer.Transform(dataView);

    // 'preparedData' is a 'promise' of data, lazy-loading. call Preview  
    //and iterate through the returned collection from preview.

    var preViewpreparedData = preparedData.Preview(maxRows: numberOfRows);

    foreach (var row in preViewpreparedData.RowView)
    {
        var ColumnCollection = row.Values;
        string lineToPrint = "Row--> ";
        foreach (KeyValuePair<string, object> column in ColumnCollection)
        {
            lineToPrint += $"| {column.Key}:{column.Value}";
        }
        Console.WriteLine(lineToPrint + "\n");
    }
}

In [9]:
IEstimator<ITransformer> estimator = mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "PersonnelNumberEncoded", inputColumnName: "PersonnelNumber")
    .Append(mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "SkillIdEncoded", inputColumnName: "SkillId"));
PeekDataViewInConsole(mlContext, trainLoader, estimator, 10);

var options = new MatrixFactorizationTrainer.Options
{
    MatrixColumnIndexColumnName = "PersonnelNumberEncoded",
    MatrixRowIndexColumnName = "SkillId",
    LabelColumnName = "SkillLevel",
    NumberOfIterations = 200,
    ApproximationRank = 100
};

//var trainerEstimator = estimator.Append(mlContext.Recommendation().Trainers.MatrixFactorization(options));

//var trainerEstimator = estimator.Append(mlContext.BinaryClassification.Trainers.FieldAwareFactorizationMachine(new string[] { "SkillId" }));
//var model = estimator.Fit(trainLoader);

var trainerEstimator = estimator.Append(mlContext.Recommendation().Trainers.MatrixFactorization(options));
ITransformer model = estimator.Fit(trainLoader);


 
Peek data in DataView: Showing 10 rows with the columns
#######################################################
Row--> | PersonnelNumber:E00267| SkillId:14| SkillLevel:3| PersonnelNumberEncoded:1| SkillIdEncoded:1

Row--> | PersonnelNumber:E00267| SkillId:47| SkillLevel:3| PersonnelNumberEncoded:1| SkillIdEncoded:2

Row--> | PersonnelNumber:E00267| SkillId:50| SkillLevel:3| PersonnelNumberEncoded:1| SkillIdEncoded:3

Row--> | PersonnelNumber:E00267| SkillId:56| SkillLevel:3| PersonnelNumberEncoded:1| SkillIdEncoded:4

Row--> | PersonnelNumber:E00267| SkillId:137| SkillLevel:3| PersonnelNumberEncoded:1| SkillIdEncoded:5

Row--> | PersonnelNumber:E00267| SkillId:139| SkillLevel:3| PersonnelNumberEncoded:1| SkillIdEncoded:6

Row--> | PersonnelNumber:E00267| SkillId:191| SkillLevel:3| PersonnelNumberEncoded:1| SkillIdEncoded:7

Row--> | PersonnelNumber:E00267| SkillId:265| SkillLevel:3| PersonnelNumberEncoded:1| SkillIdEncoded:8

Row--> | PersonnelNumber:E00267| SkillId:337| SkillLevel:3

In [15]:
Color color = Color.FromArgb(130,150,115);

var prediction = model.Transform(testLoader);

var metrics = mlContext.Regression.Evaluate(prediction, labelColumnName: "Label", scoreColumnName: "Score");

Console.WriteLine(prediction.GetRowCount());
Console.WriteLine(prediction.Schema[0]);

var metrics = mlContext.Regression.Evaluate(prediction, labelColumnName: "SkillLevel", scoreColumnName: "Score");
//var metrics = mlContext.BinaryClassification.Evaluate(data: prediction, labelColumnName: "PersonnelNumberEncoded", predictedLabelColumnName: "SkillId");
//Console.WriteLine("Evaluation Metrics: acc:" + Math.Round(metrics.Accuracy, 2) + " AreaUnderRocCurve(AUC):" + Math.Round(metrics.AreaUnderRocCurve, 2),color);
            
Console.WriteLine("Root Mean Squared Error : " + metrics.RootMeanSquaredError.ToString());
Console.WriteLine("RSquared: " + metrics.RSquared.ToString());


PersonnelNumber: String


Unhandled exception: System.ArgumentOutOfRangeException: Score column 'Score' not found (Parameter 'schema')
   at Microsoft.ML.Data.RoleMappedSchema.MapFromNames(DataViewSchema schema, IEnumerable`1 roles, Boolean opt)
   at Microsoft.ML.Data.RoleMappedSchema..ctor(DataViewSchema schema, IEnumerable`1 roles, Boolean opt)
   at Microsoft.ML.Data.RoleMappedData..ctor(IDataView data, Boolean opt, KeyValuePair`2[] roles)
   at Microsoft.ML.Data.RegressionEvaluator.Evaluate(IDataView data, String label, String score)
   at Microsoft.ML.RegressionCatalog.Evaluate(IDataView data, String labelColumnName, String scoreColumnName)
   at Submission#19.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)